In [1]:
from utils import FakeTRTFunc
from model import load_as_trt_model
import chess.syzygy as syzygy

use_fake = True
if use_fake:
    trt_func = FakeTRTFunc()
else:
    trt_func, _ = load_as_trt_model()
tablebase = syzygy.open_tablebase("/home/tomaz/ChessBot_v2/syzygy/3-4-5")

2025-01-21 18:57:19.236216: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from selfplay import play_game
from configs import selfplayConfig

_, (_, _) = play_game(trt_func, tablebase, 2)



  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 8
  +---+---+---+---+---+---+---+---+
  | k | Q |   |   |   |   |   |   | 7
  +---+---+---+---+---+---+---+---+
  | B |   |   |   |   |   |   |   | 6
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 5
  +---+---+---+---+---+---+---+---+
  | p | R | P |   |   |   |   | P | 4
  +---+---+---+---+---+---+---+---+
  | P |   | K |   |   |   |   | P | 3
  +---+---+---+---+---+---+---+---+
  |   |   |   | p |   |   |   |   | 2
  +---+---+---+---+---+---+---+---+
  | R | N |   |   |   |   |   |   | 1
  +---+---+---+---+---+---+---+---+
    a   b   c   d   e   f   g   h

Fen: 8/kQ6/B7/8/pRP4P/P1K4P/3p4/RN6 b - - 1 87
Key: 4cf841524c3e81d5
White wins by checkmate in 173 moves in 17.26 seconds


ValueError: not enough values to unpack (expected 3, got 2)

In [1]:
from rchess import Board
board = Board("r3r3/2P4k/3Bbbqp/ppQ2pp1/4pPP1/1P6/P1R2N1P/3R2K1 w - -")
print(board)
print(board.legal_moves())


  +---+---+---+---+---+---+---+---+
  | r |   |   |   | r |   |   |   | 8
  +---+---+---+---+---+---+---+---+
  |   |   | P |   |   |   |   | k | 7
  +---+---+---+---+---+---+---+---+
  |   |   |   | B | b | b | q | p | 6
  +---+---+---+---+---+---+---+---+
  | p | p | Q |   |   | p | p |   | 5
  +---+---+---+---+---+---+---+---+
  |   |   |   |   | p | P | P |   | 4
  +---+---+---+---+---+---+---+---+
  |   | P |   |   |   |   |   |   | 3
  +---+---+---+---+---+---+---+---+
  | P |   | R |   |   | N |   | P | 2
  +---+---+---+---+---+---+---+---+
  |   |   |   | R |   |   | K |   | 1
  +---+---+---+---+---+---+---+---+
    a   b   c   d   e   f   g   h

Fen: r3r3/2P4k/3Bbbqp/ppQ2pp1/4pPP1/1P6/P1R2N1P/3R2K1 w - - 0 1
Key: 5a9e3985f34193a2
["g1f1", "g1h1", "g1g2", "f2e4", "f2h1", "f2d3", "f2h3", "c5a3", "c5e3", "c5b4", "c5d4", "c5b6", "c5a7", "d6e5", "d6e7", "d6f8", "d1a1", "d1b1", "d1c1", "d1e1", "d1f1", "d1d2", "d1d3", "d1d4", "d1d5", "c2c1", "c2b2", "c2d2", "c2e2", "c2c3", "c2c4", "

In [ ]:
from selfplay import run_selfplay

run_selfplay(
    num_agents=1,
    num_games=10,
    buffer_size=64,
    use_fake_model=True
)

KeyboardInterrupt: 

Process Process-4:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/tomaz/ChessBot_v2/python/selfplay/__init__.py", line 60, in play_games
    games_count.value += 1
  File "/usr/lib/python3.10/multiprocessing/managers.py", line 1137, in get
    return self._callmethod('get')
  File "/usr/lib/python3.10/multiprocessing/managers.py", line 817, in _callmethod
    conn.send((self._id, methodname, args, kwds))
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 206, in send
    self._send_bytes(_ForkingPickler.dumps(obj))
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 411, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Err